## Proceso de ETL en el dataset output_steam_games 

#### Este proceso se realiza con la finalidad de leer el dataset con el formato correcto, eliminar las columnas que no se necesitan para responder las consultas y hacer las transformaciones necesarias para optimizar el rendimiento de la API y el entrenamiento del modelo de aprendizaje automático.

### Se importan las librerías a utilizar en el proceso de ETL.

In [19]:
import pandas as pd
import json
import pyarrow

### El archivo JSON se carga en un DataFrame utilizando la función read_json() de pandas.

In [2]:
df_games = pd.read_json('datasets_originales\output_steam_games.json', orient='records', lines=True, convert_dates=True)

### Visualización del DataFrame.

In [6]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


### Se eliminan los valores faltantes del DataFrame. 

In [7]:
df_games = df_games.dropna() 

### Se visualizan las columnas del DataFrame para descartar las columnas que no son necesarias.

In [8]:
df_games.columns 

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

### Se eliminan columnas específicas del DataFrame con el objetivo de reducir su tamaño y mejorar su eficiencia.

In [9]:
df_games = df_games.drop(['reviews_url','url','tags', 'app_name', 'specs', 'price', 'early_access'], axis=1)

### Se visualiza el DataFrame para confirmar si muestra los resultados deseados. 
#### Nota: hasta este punto se decide mantener las columnas "publisher" y "developer" ya que ambas contienen datos similares y posiblemente alguna pueda ser el complemento de la otra.

In [10]:
df_games.head()

,publisher,genres,title,release_date,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,767400.0,彼岸领域
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,772540.0,Trickjump Games Ltd


### Se convierte la lista de géneros a una cadena única separada por comas.

In [11]:
df_games['genres'] = df_games['genres'].apply(lambda x: ', '.join(x))  

## Proceso de creación de variables dummy para representar las variables categóricas de la columna "genres"
### Para que los algoritmos de aprendizaje automático puedan trabajar con estos datos, se requiere convertir la columna "genres" en una forma numérica.

In [12]:
genres_dummy = df_games['genres'].str.get_dummies(', ')

### Se concatenan las variables dummy con el DataFrame original.

In [13]:
df_games_dummy = pd.concat([df_games, genres_dummy], axis=1)

### Se extrae el año de la columna "release_date" y se crea la columna "year"


In [14]:
df_games = df_games_dummy 
# Reemplazar los valores no válidos por NaN / Replace invalid values ​​with NaN
df_games["release_date"] = pd.to_datetime(df_games["release_date"], errors='coerce')
# Filtrar el DataFrame para obtener los registros válidos / Filter the DataFrame to get valid records
df_filtered = df_games.dropna(subset=["release_date"])
# Convertir la columna "release_date" a tipo datetime / Convert "release_date" column to datetime type
df_filtered["release_date"] = pd.to_datetime(df_filtered["release_date"])
# Rellenando los valores faltantes en las columna'release_date' con la moda / Filling missing values ​​in 'release_date' columns with mode
columns_fill_mode = ['release_date']
for col in columns_fill_mode:
    df_games[col].fillna(df_games[col].mode()[0], inplace=True)
# Modelado y eliminación de la columna 'release_date' / Modeling and removing 'release_date' column
df_games['year'] = df_games['release_date'].dt.year.astype('Int64')
df_games.drop('release_date', axis=1, inplace=True)

C:\Users\58424\AppData\Local\Temp\ipykernel_10572\638431184.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["release_date"] = pd.to_datetime(df_filtered["release_date"])


In [15]:
df_games.head(2)

,publisher,genres,title,id,developer,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,...,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,year
88310,Kotoshiro,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,761140.0,Kotoshiro,1,0,0,0,1,...,0,0,1,0,0,1,0,0,0,2018
88311,"Making Fun, Inc.","Free to Play, Indie, RPG, Strategy",Ironbound,643980.0,Secret Level SRL,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,2018


### El DataFrame obtenido de los diferentes procesos se exporta a un archivo parquet.

In [18]:
df_games.to_parquet('output_steam_games_clean.parquet.gz', compression='gzip', index=False) 